SalesLT_SalesOrderHeader

In [0]:
# Enable auto merge

spark.sql("SET spark.databricks.delta.schema.autoMerge.enabled = true")



In [0]:
from pyspark.sql import DataFrame, Window
from pyspark.sql import functions as F
from pyspark.sql.types import (
    DecimalType, IntegerType, StringType, TimestampType, StructType, StructField, BooleanType
)
from pyspark.sql.functions import col, count, when, datediff, current_date, desc

In [0]:
%run "/Workspace/Utils/Utils"

In [0]:
#Loading Table

df = spark.table("adlslmcompany_bronze.managed_bronze.saleslt_salesorderheader")

In [0]:
df.display()

In [0]:
#Checking for duplicated ID's

checkduplicates(df, "SalesOrderID" )

In [0]:
# Checking for outliers in numerical columns 

In [0]:

iqr_outlier(df, "SubTotal") 

# Although the value identified by the IQR outlier function is considered an outlier, it is not an error and should be retained. If necessary, it may be addressed in the gold column depending on the intended use of the table.

In [0]:
iqr_outlier(df, "TaxAmt") 

#Although the value identified by the IQR outlier function is considered an outlier, it is not deemed an error and should be retained. If necessary, it may be addressed in the gold column based on the intended use of the table.

In [0]:
iqr_outlier(df, "Freight") 

#Although the value identified by the IQR outlier function is considered an outlier, it is not classified as an error and should be retained. If necessary, it may be addressed in the gold column based on the intended use of the table.

In [0]:
iqr_outlier(df, "TotalDue") 


#Although the value identified by the IQR outlier function is considered an outlier, it is not classified as an error and should be retained. If necessary, it may be addressed in the gold column based on the intended use of the table.

In [0]:
# It is necessary to verify if there are any distinct values for the Online Order Flag that are not null. After consultation with the responsible party, it has been confirmed that this column is still in use and should not be deleted.

df.groupBy("OnlineOrderFlag").count().display()

In [0]:
# It is necessary to verify if there are any non-null values in the Credit Card Approval Code. If all values are null, and it has been confirmed through a meeting with the responsible party that this column is no longer in use, it should be deleted.

df.groupBy("CreditCardApprovalCode").count().display()

In [0]:
def silver_clean_orderheader(df): 


    #Deleting ireelevant columns
    df = df.drop("RevisionNumber",  "SalesOrderNumber", "PurchaseOrderNumber", "AccountNumber", "Comment" )

    #Time to ship
    df = df.withColumn(
        "Shiptime", 
        when(
            col("ShipDate").isNull(), 
            datediff(current_date(), col("OrderDate"))
        ).otherwise(
            datediff(col("Shipdate"), col("OrderDate"))
        ).cast(IntegerType())
    )


    # Adds transformation Date column
    df = df.withColumn("silves_transformed_timestamp", F.current_timestamp())


        #Cast to ensure datatype
    df = df.select(
         F.col('SalesOrderID').cast(IntegerType()).alias('SalesOrderID'),
         F.col('OrderDate').cast(TimestampType()).alias('OrderDate'),
         F.col('DueDate').cast(TimestampType()).alias('DueDate'),
         F.col('ShipDate').cast(TimestampType()).alias('ShipDate'),
         F.col('Status').cast(IntegerType()).alias('Status'),
         F.col('OnlineOrderFlag').cast(BooleanType()).alias('OnlineOrderFlag'),
         F.col('ShipToAddressID').cast(IntegerType()).alias('ShipToAddressID'),
         F.col('BillToAddressID').cast(IntegerType()).alias('BillToAddressID'),
         F.col('ShipMethod').cast(StringType()).alias('ShipMethod'),
         F.col('CreditCardApprovalCode').cast(StringType()).alias('CreditCardApprovalCode'),
         F.col('SubTotal').cast(DecimalType(19,4)).alias('SubTotal'),
         F.col('TaxAmt').cast(DecimalType(19,4)).alias('TaxAmt'),
         F.col('Freight').cast(DecimalType(19,4)).alias('Freight'),
         F.col('TotalDue').cast(DecimalType(19,4)).alias('TotalDue'),
         F.col('Shiptime').cast(IntegerType()).alias('Shiptime'),
         F.col('CustomerID').cast(IntegerType()).alias('CustomerID'),
         F.col('rowguid').cast(StringType()).alias('rowguid'),
         F.col('ModifiedDate').cast(TimestampType()).alias('ModifiedDate'),
         F.col('bronze_ingestion_timestamp').cast(TimestampType()).alias('bronze_ingestion_timestamp'),
         F.col('silves_transformed_timestamp').cast(TimestampType()).alias('silves_transformed_timestamp'),
                 )
    

    return df

In [0]:
#Defining expected schema
expected_schema = StructType([
    StructField("SalesOrderID", IntegerType(), False),   
    StructField("OrderDate", TimestampType(), False),
    StructField("DueDate", TimestampType(), False),
    StructField("ShipDate", TimestampType(), False),
    StructField("Status", IntegerType(), False),
    StructField("OnlineOrderFlag", BooleanType(), False),
    StructField("ShipToAddressID", IntegerType(), False),
    StructField("BillToAddressID", IntegerType(), False),
    StructField("ShipMethod", StringType(), False),
    StructField("CreditCardApprovalCode", StringType(), False),
    StructField("SubTotal", DecimalType(19,4), False),
    StructField("TaxAmt", DecimalType(19,4), False),
    StructField("Freight", DecimalType(19,4), False),
    StructField("TotalDue", DecimalType(19,4), False),
    StructField("Shiptime", IntegerType(), False),
    StructField("CustomerID", IntegerType(), False),
    StructField("rowguid", StringType(), False),
    StructField("ModifiedDate", TimestampType(), False),
    StructField("bronze_ingestion_timestamp", TimestampType(), False),
    StructField("silves_transformed_timestamp", TimestampType(), False)
    ])


In [0]:
#Transforming df

silver_df = silver_clean_orderheader(df)

In [0]:
#Checking the schema 
_validate_schema(silver_df, expected_schema)

In [0]:
#Comparing lenghts

compare_lengths(df, silver_df)

**IMPORTANT: Please note that this is a simulated project; the upsert operation will be executed within this notebook. In a production environment, a dedicated notebook containing only the function and validations would be developed. All function notebooks would be orchestrated by Azure Data Factory (ADF) pipelines or Azure Databricks (ADB) workflows. The method of upsert may vary based on the utilization of auto loader, streaming, or Change Data Feed (CDF).**

In [0]:
#Loading into the Silver Layer     

target_table= "saleslt_orderheader"   

schema = "managed_silver"

catalog = "adlslmcompany_silver"

primary_keys = ["SalesOrderID"]


_upsert_silver_table(silver_df, target_table, primary_keys, schema, catalog )